In [1]:
import geopandas as gpd
import gdal
from osgeo import ogr

from shapely.geometry import Polygon, LineString, MultiLineString
%matplotlib inline

In [2]:
def coverToLine(shpapeFileInput):
    adminBoundary = gpd.read_file(shpapeFileInput)
    for i, name in enumerate(adminBoundary.geom_type):
        if name == 'Polygon':
            adminBoundary['geometry'][i] = LineString((adminBoundary['geometry'][i].exterior.coords))
        if name == 'MultiPolygon':
            polylist = []
            for singlepoly in adminBoundary['geometry'][i]:
                polylist.append(singlepoly.exterior.coords)
            adminBoundary['geometry'][i] = MultiLineString(polylist)
                
    #adminBoundary['geometry'] = adminBoundary['geometry'].buffer(2.0)
    adminBoundary.to_file("temp.shp")


In [3]:
def convertTo4326(shpapeFileInput):
    adminBoundary = gpd.read_file(shpapeFileInput)
    adminBoundary.to_crs(epsg = 4326, inplace= True)
    adminBoundary.to_file("temp.shp")    

In [4]:
def pts2raster_without_tiff_file_input(shpapeFileInput,rasterFileOut,cellSize,field_name=False):
    
    # Define pixel_size of new raster
    #pixel_size determines the size of the new raster.
    #pixel_size is proportional to size of shapefile.
    pixel_size = cellSize
    
    #define NoData value of new raster
    NoData_value = -9999
    
    # input shape/OGR filem => shpapeFileInput
    
    # name of the raster Tiff that will be created =>  rasterFileOut
 
    #admin boundary shape file consist polygon/multipolygon geometry
    #if we rasterizes this shape file as it is we will get black tiff file
    #so to avoid this we need to convert polygon/multipolygon geometry to line geometry
    coverToLine(shpapeFileInput)
    
    #convert to EPSG-4326
    convertTo4326("temp.shp")
    # Open the data source and read in the extent
    source_ds = ogr.Open("temp.shp")
    
    #getting layer information of shapefile.
    source_layer = source_ds.GetLayer()
    
    #get extent values to set size of output raster.
    x_min, x_max, y_min, y_max = source_layer.GetExtent()
    print(int((x_max - x_min)))
    print(int((y_max - y_min)))
    
    # Create the destination data source
    
    # Calculate size/resolution of the raster.
    x_res = int((x_max - x_min) / pixel_size)
    y_res = int((y_max - y_min) / pixel_size)
    #x_res = y_res = 0.000277
    print(x_res)
    print(y_res)
    
    #get GeoTiff driver by 
    drv_tiff = gdal.GetDriverByName('GTiff')

    #passing the filename, x and y direction resolution, no. of bands.
    target_ds = drv_tiff.Create(rasterFileOut, x_res, y_res, 1, gdal.GDT_Int16 )

    #transforms between pixel raster space to projection coordinate space
    target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
    
    #get required raster band.
    band = target_ds.GetRasterBand(1)
    
    #assign no data value to empty cells.
    band.SetNoDataValue(NoData_value)
    
    #main conversion method
    if field_name:
        gdal.RasterizeLayer(target_ds,[1], source_layer,options=["ATTRIBUTE={0}".format(field_name)])
    else:
        gdal.RasterizeLayer(target_ds,[1], source_layer,burn_values=[-1])

In [5]:
pts2raster_without_tiff_file_input("Uganda_Parish.shp","Uganda_Parish.tif", 0.000277) #0.00027

5
5
19598
20625


In [6]:
pts2raster_without_tiff_file_input("Rwanda_Village_Boundaries.shp","Rwanda_Village_Boundaries.tif", 0.000277)

2
1
7355
6471


In [7]:
pts2raster_without_tiff_file_input("Ethiopia_AdminBoundaries.shp","Ethiopia_AdminBoundaries.tif", 0.000277)

10
11
36124
41084
